In [61]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix

In [38]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [39]:
data.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [40]:
data.dtypes

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [41]:
# Encoding the data:
data = data.iloc[:,3:]
data['Gender'] = data['Gender'].map({'Female':0, 'Male':1})
data = pd.get_dummies(data) # onehot encoding

In [42]:
# Generating Matrix Features

X = data.drop(axis=1, columns=['Exited'])
Y = data['Exited']

In [44]:
# Splitting dataset into training and testing dataset

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=101)

In [45]:
# perform the scaling operation:

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.fit_transform(x_test)

In [46]:
# Initializing Artificial Neural Network

ann = tf.keras.models.Sequential()

# creating multiple hidden layers:
ann.add(tf.keras.layers.Dense(units=16,activation="relu"))
ann.add(tf.keras.layers.Dense(units=6,activation="relu"))
ann.add(tf.keras.layers.Dense(units=10,activation="relu"))
ann.add(tf.keras.layers.Dense(units=6,activation="relu"))

# adding output layers:
ann.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))  # here units:1, becz binary classification

# compiling the Artificial neural Network (using adam optimizer):
ann.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

In [47]:
# Fitting the Network:

ann.fit(x_train_scaled, y_train, batch_size=32, epochs=100)

Epoch 1/100
219/219 [==============================] - 2s 2ms/step - loss: 0.5915 - accuracy: 0.7311
Epoch 2/100
219/219 [==============================] - 0s 2ms/step - loss: 0.4506 - accuracy: 0.7990
Epoch 3/100
219/219 [==============================] - 0s 2ms/step - loss: 0.4298 - accuracy: 0.8100
Epoch 4/100
219/219 [==============================] - 0s 2ms/step - loss: 0.4181 - accuracy: 0.8207
Epoch 5/100
219/219 [==============================] - 0s 2ms/step - loss: 0.4029 - accuracy: 0.8299
Epoch 6/100
219/219 [==============================] - 0s 2ms/step - loss: 0.3779 - accuracy: 0.8433
Epoch 7/100
219/219 [==============================] - 0s 2ms/step - loss: 0.3592 - accuracy: 0.8507
Epoch 8/100
219/219 [==============================] - 0s 2ms/step - loss: 0.3544 - accuracy: 0.8544
Epoch 9/100
219/219 [==============================] - 0s 2ms/step - loss: 0.3506 - accuracy: 0.8546
Epoch 10/100
219/219 [==============================] - 0s 2ms/step - loss: 0.3479 - accura

In [70]:
# predict based on test data
# if y-pred is in between 0 to 0.5, then this new y_pred will become 0(False) & And if y_pred is larger than 0.5, then new y_pred will become 1(True).
y_pred_test = ann.predict(x_test)  
y_pred_test = y_pred_test > 0.5


# predict based on train data
y_pred_train = ann.predict(x_train > 0.5)
y_pred_train = y_pred_train > 0.5

In [71]:
y_pred_train

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [75]:
y_pred_test

array([[False],
       [ True],
       [ True],
       ...,
       [False],
       [ True],
       [ True]])

In [77]:
print("Accuracy based on train dataset: ", accuracy_score(y_train, y_pred_train))
print("Accuracy based on test dataset: ", accuracy_score(y_test, y_pred_test))
print("Confusion Matrix is: \n", confusion_matrix(y_test, y_pred_test))

Accuracy based on train dataset:  0.7921428571428571
Accuracy based on test dataset:  0.47
Confusion Matrix is: 
 [[ 955 1423]
 [ 167  455]]


In [60]:
# Finally save the model:

ann.save("ANN.h5")